In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

Install OR-Tools for Python

In [ ]:
python -m pip install --upgrade --user ortools

In [ ]:
yyy= {1: [(9, 1, (1,)), (10, 1, (1,))],
 2: [(1, 1, (1, 2, 3, 4, 5, 6)),
  (2, 2, (1, 2, 3, 4, 5, 6)),
  (3, 2, (1, 2, 3, 4, 5, 6)),
  (4, 2, (1, 2, 3, 4, 5, 6)),
  (5, 3, (2, 4, 6)),
  (6, 3, (2, 4, 6)),
  (7, 2, (3, 6)),
  (8, 3, (2, 4, 6))]}

In [ ]:
# # Routing Google

# In[37]:


def affectdata(yyy):
    affect={}
    for clu in yyy.keys():
        #print(clu)
        affect[clu]={}
        for v in range(1,vs+1):
            #print("vsssssss",vs)
            affect[clu][v]={}
            for d in range(1,7):
                affect[clu][v][d]=[]
                for i in range(0,len(yyy[clu])):##################### ####################### a supprimer le vide
                        #print(yyy[clu][i])
                        if ((yyy[clu][i][1]==v) and d in yyy[clu][i][2]):

                            affect[clu][v][d].append(yyy[clu][i][0])
    

    for clu in yyy.keys():

        for v in range(1,vs+1):
            if affect[clu][v]=={}:
                    del affect[clu][v]
            for d in range(1,7):
                if affect[clu][v][d]==[]:
                    del affect[clu][v][d]
    
    return(affect)

def googlealldata(yyy,affect,vrp,locali_x,locali_y):
    alldata={}
    data={}
    for clu in affect.keys():
        alldata[clu]={}
        for v in range(1,vs+1):
            if v in affect[clu].keys():
                
                alldata[clu][v]={}
                alldata[clu][v]['locations']={}
                #print(affect[clu])
                for i in affect[clu][v]:#enumeration des patients

                    alldata[clu][v]['locations'][i]=[]
                    alldata[clu][v]['locations'][i].append((locali_x[0],locali_y[0]))

                    for p in affect[clu][v][i]:
                        #print( vrp[vrp.clusters==clu]['y'].to_dict()[p])
                        alldata[clu][v]['locations'][i].append((vrp[vrp.clusters==clu]['x'].to_dict()[p],vrp[vrp.clusters==clu]['y'].to_dict()[p])) #recupération de positions des patients
                    #print(alldata[clu][v])


    print (alldata)
    return(alldata)

def create_data_model(dataOneDay):
    """Stores the data for the problem."""
    #data['locations']=[((loc_x[0],loc_y[0]))]
    #for i in range(1,len(loc_x)):
     #   data['locations'].append((loc_x[i],loc_y[i]))
    #print (data)
    data={}
    data['locations'] = dataOneDay['locations']
    data['num_vehicles'] = 1
    data['depot'] = 0
    #print(data)
    return data
def compute_euclidean_distance_matrix(locations):
    """Creates callback to return distance between points."""
    distances = {}
    for from_counter, from_node in enumerate(locations):
        distances[from_counter] = {}
        for to_counter, to_node in enumerate(locations):
            if from_counter == to_counter:
                distances[from_counter][to_counter] = 0
            else:
                # Euclidean distance
                distances[from_counter][to_counter] = round(int(

                    hypot((from_node[0] - to_node[0]),
                               (from_node[1] - to_node[1]))),3)
    return distances

def print_solution(manager, routing, solution,V):

    """Prints solution on console."""
    print('Objective: {}'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = ''

    route_distance = 0
    while not routing.IsEnd(index):
        #print(manager.IndexToNode(index))#
        if (manager.IndexToNode(index) == 0):
            #print(V[manager.IndexToNode(index)] )#
            plan_output += ' {} ->'.format(manager.IndexToNode(index))
            solList=[(manager.IndexToNode(index))]
        else:
            plan_output += ' {} ->'.format(V[manager.IndexToNode(index) - 1])
            solList.append(V[manager.IndexToNode(index)-1])

        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    ch2 = "".join(reversed(plan_output))

    #print(ch2)

    solList.append(0)

    #[::-1]
    plan_output += 'Objective: {}m\n'.format(route_distance)

    return(solList,route_distance)

def resultrouting(alldata):
    print("*************** all data *******************")
#    print(alldata)
    print("**********************************")
    solutionsGoogle={}
    objGoogle=0
    for clu in alldata.keys():
        solutionsGoogle[clu]={}
        print("*******************Cluster",clu,"**************************")
        for v in alldata[clu].keys():
            print("-------------Cluster",clu,"----Vehicule",v,"------------")
            if (alldata[clu][v]['locations'] != {}):
                solutionsGoogle[clu][v]={}
                for i in alldata[clu][v]['locations']:
                    print("Tour de jour",i)
                   #for i in range(1,len(alldata['locations'])+1):
                    #print(alldata['locations'][i])
                    dataoneday={}
                    dataoneday['locations']=alldata[clu][v]['locations'][i]
                    #print(dataoneday)
                    # Instantiate the data problem.
                    data = create_data_model(dataoneday)
                    if data:

                        # Create the routing index manager.
                        manager = pywrapcp.RoutingIndexManager(len(data['locations']),
                                                               data['num_vehicles'], data['depot'])

                        # Create Routing Model.
                        routing = pywrapcp.RoutingModel(manager)

                        distance_matrix = compute_euclidean_distance_matrix(data['locations'])

                        def distance_callback(from_index, to_index):
                            """Returns the distance between the two nodes."""
                            # Convert from routing variable Index to distance matrix NodeIndex.
                            from_node = manager.IndexToNode(from_index)
                            to_node = manager.IndexToNode(to_index)
                            return distance_matrix[from_node][to_node]

                        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

                        # Define cost of each arc.
                        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

                        # Setting first solution heuristic.
                        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
                        search_parameters.first_solution_strategy = (
                            routing_enums_pb2.FirstSolutionStrategy.GLOBAL_CHEAPEST_ARC)
#                            routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
#                            routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH)
#                        search_parameters.time_limit.seconds = 20
#                        search_parameters.log_search = True
                        #PATH_CHEAPEST_ARC

                        # Solve the problem.
                        solution = routing.SolveWithParameters(search_parameters)

                        # Print solution on console.
                        if solution:
                            sol, obj= print_solution(manager, routing, solution,affect[clu][v][i])
                            objGoogle+=obj
                            solutionsGoogle[clu][v][i]=sol

    print("solutionsGoogle",solutionsGoogle)
    print("objGoogle",objGoogle)

    return(solutionsGoogle,objGoogle)

def affectjours(solutionsGoogle):
    lun = {}
    mar ={}
    merc ={}
    jeu = {}
    vend = {}
    sam ={}
    for clu in solutionsGoogle.keys(): #pour chaque cluster
        lun[clu] = []      #creer une liste pour chaque jour
        mar[clu] = []
        merc[clu] = []
        jeu[clu] = []
        vend[clu] = []
        sam[clu] = []
        for v in range(1,vs+1):

            if v in solutionsGoogle[clu].keys() and solutionsGoogle[clu][v] != []:
                for k in solutionsGoogle[clu][v]:    #pour
                    print(k)
                    if(k==1):
                        lun[clu].append(solutionsGoogle[clu][v][k]) #append ij k
                    elif(k==2):
                        mar[clu].append(solutionsGoogle[clu][v][k])
                    elif(k==3):
                        merc[clu].append(solutionsGoogle[clu][v][k])
                    elif(k==4):
                        jeu[clu].append(solutionsGoogle[clu][v][k])
                    elif(k==5):
                        vend[clu].append(solutionsGoogle[clu][v][k])
                    else:
                        sam[clu].append(solutionsGoogle[clu][v][k])
    return(lun,mar,merc,jeu,vend,sam)

affect=affectdata(yyy)

alldata = googlealldata(yyy,affect,vrp,locali_x,locali_y)
print (alldata)
solutionsGoogle,objGoogle =resultrouting(alldata)